# 채팅 서버/클라이언트 구현하기

* 난이도 : ★★★★★☆☆☆☆☆
* 필요라이브러리: socket, socketserver, threading


* 간단한 채팅 서버와 클라이언트를 구현해봅니다.
* 서버구현시 네트워크 프레임워크인 socketserver 라이브러리를 사용해봅니다.
* socketserver 공식문서 링크: https://docs.python.org/3/library/socketserver.html

In [ ]:
import socketserver

# socketserver.BaseRequestHandler 를 상속받아 새로운 핸들러 클래스를 생성합니다.
class MyHandler(socketserver.BaseRequestHandler):
    # BaseReqeustsHandler 에 있는 handle 함수를 오버라이딩 해서 내가 원하는 기능으로 구현할 예정입니다.
    def handle(self):
        print(self.client_address)

# socketserver.ThreadingMixIn 과 socketserver.TCPServer 를 믹스인 하여 ChatServer 클래스를 선언했습니다.
# 여기서 필요한건 믹스인된 클래스 자체가 필요하고 실제 기능은 따로 구현할게 없어 pass 했습니다.
class ChatServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

# 믹스인해서 새롭게 생성된 ChatServer 클래스를 생성합니다.
# 실제 socketserver.py 파일의 TCPServer 생성자를 보면
# server_address, RequestsHandlerClass 인자가 필수 인자로 요구하고 있습니다.
server = ChatServer(("", 12000), MyHandler)

# 서버를 구동합니다.
server.serve_forever()

# 구동이 끝나면 종료합니다.
server.shutdown()

# 종료 후 서버를 닫습니다.
server.server_close()

### 몇줄 안되는 코드지만 어려운 코드
위 코드는 주석을 제거하면 실제 10줄 밖에 안되는 짧은 코드입니다만 채팅서버가 동작하는데 필요한 필수적인 기능은 이미 구현이 끝난 상태입니다. 해당 코드를 모두 이해하려면 상당히 많은 사전지식이 있어야 합니다. 이전의 강좌를 참고하시고 그래도 모르겠다 싶으시면 구글링도 많이 해보시고 그래도 모르겠다 싶으시면 그냥 따라만 하시고 넘어가시길 바랍니다.


이제 여기서 handle() 함수와 MyHandler 클래스에 필요한 기능을 추가하면 됩니다.

### 채팅 서버 시나리오

1. 접속이 되면 사용자에게 닉네임을 입력받습니다. 닉네임은 중복되지 않아야 합니다.
2. 닉네임 입력이 끝나면 해당 사용자에 대한 정보를 서버쪽에서 기억하고 있어야 합니다.
3. 유저 추가가 끝나면 사용자가 보내는 채팅 데이터를 계속 받아서 다른 사용자에게 채팅내용을 알려줘야 합니다.
4. 사용자가 종료되면 서버쪽에 등록된 유저에서 삭제하는 기능도 필요합니다.

In [ ]:
'''
chat_server.py
'''
import socketserver

# socketserver.BaseRequestHandler 를 상속받아 새로운 핸들러 클래스를 생성합니다.
class MyHandler(socketserver.BaseRequestHandler):
    '''socketserver.BaseRequestHandler를 상속받아 만든 클래스
    (이 클래스는 믹스인으로 생성된 ChatServer 클래스의 인자로 들어갑니다.)'''
    # 현재 접속된 유저를 관리할 변수
    users = {}
    
    def send_to_all(self, msg):
        '''채팅 참여중인 전체 인원에게 메세지를 보내는 함수
        
        Args:
            msg (str) : 메세지
        
        Returns:
            Nothing
        '''
        for sock, _ in self.users.values():
            sock.send(msg.encode())
            
    def handle(self):
        '''BaseRequestsHandler 에 있는 handle 함수를 오버라이딩 한 함수'''
        print(self.client_address)
        
        # 닉네임이 중복되지 않으면 무한루프 탈출
        while True:
            # 접속된 유저에게 메세지를 보냅니다.
            # 여기서 request 는 BaseRequestsHandler 에 있는 소켓 객체 입니다.
            self.request.send("채팅 닉네임을 입력하세요.: ".encode())
            
            # 접속된 유저로부터 데이터를 1024 크기만큼 수신합니다.
            # 소켓은 bytes 형태로 데이터가 송수신 되기 때문에
            # str 형태를 보낼때는 encode() 받을때는 decode() 해줘야 합니다.
            nickname = self.request.recv(1024).decode()
            
            # 접속된 유저에게 입력받은 nickname 이 존재하는지 확인
            if nickname in self.users:
                self.request.send("이미 등록된 닉네임 입니다.\n".encode())
            else:
                # 중복되지 않은 닉네임인 경우 접속된 소켓과 주소를 튜플형태로
                # users 에 nickname 을 키값으로 저장합니다.
                self.users[nickname] = (self.request, self.client_address)
                
                # 서버 로그
                print("현재 {} 명 참여중".format(len(self.users)))
                
                # 접속중인 모든 사용자에게 신규 입장을 알립니다.
                self.send_to_all("[{}]님이 입장 했습니다.".format(nickname))
                break

        # 채팅 데이터를 수신하고 다른 사용자에게 전달하는 무한 루프
        while True:
            # 접속된 사용자로부터 데이터가 수신되면
            msg = self.request.recv(1024)
            # 수신된 데이터가 /bye 이면 접속 종료
            if msg.decode() == "/bye":
                self.request.close()
                break
            # 수신된 데이터를 접속중인 전체 사용자에게 전송
            # [홍길동] 안녕 이런 형태로 보냄
            self.send_to_all("[{}] {}".format(nickname, msg.decode()))
            
        # 위의 while문을 빠져나왔다는건 접속종료되었다는 의미
        # 혹시 모를 오류 방지를 위해 self.users 에 존재하는지 한번 더 확인
        if nickname in self.users:
            # 접속중인 목록에서 해당 닉네임키를 삭제
            del self.users[nickname]
            
            # 현재 접속중 사용자들에게 알림 전송
            self.send_to_all("[{}] 님이 퇴장하셨습니다.".format(nickname))
            
            # 서버로그
            print("현재 {} 명 참여중".format(len(self.users)))
        
# socketserver.ThreadingMixIn 과 socketserver.TCPServer 를 믹스인 하여 ChatServer 클래스를 선언했습니다.
# 여기서 필요한건 믹스인된 클래스 자체가 필요하고 실제 기능은 따로 구현할게 없어 pass 했습니다.
class ChatServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

# 믹스인해서 새롭게 생성된 ChatServer 클래스를 생성합니다.
# 실제 socketserver.py 파일의 TCPServer 생성자를 보면
# server_address, RequestsHandlerClass 인자가 필수 인자로 요구하고 있습니다.
server = ChatServer(("", 12000), MyHandler)

# 서버를 구동합니다.
server.serve_forever()

# 구동이 끝나면 종료합니다.
server.shutdown()

# 종료 후 서버를 닫습니다.
server.server_close()

## 채팅 클라이언트 시나리오
1. 소켓을 생성하고 클라이언트 접속을 시도합니다.
2. 접속이 되면 메세지를 받을 쓰레드를 생성합니다. (입력과 받기를 동시에 할 수 없기 때문)
3. 입력문을 무한루프로 만들고 /bye 가 입력되면 종료 합니다.
4. 입력 루프를 탈출하면 소켓을 닫습니다.

In [ ]:
'''
chat_client.py
'''

# 접속에 사용할 소켓 라이브러리
import socket
# 쓰레드를 사용하기 위한 라이브러리
from threading import Thread

def recv_message(sock):
    '''서버로부터 메세지를 받을 함수
    이 함수는 쓰레드로 동작합니다.'''
    
    # 메세지는 계속 받아야 하기 때문에 무한루프
    while True:
        # 1024 크기만큼 서버로부터 데이터를 받습니다.
        msg = sock.recv(1024)
        # 화면에 출력합니다.
        print(msg.decode())

# 채팅 클라이언트용 TCP 소켓 생성
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 127.0.0.1 은 자신을 의미하는 루프백주소로 접속을 시도합니다.
sock.connect(("127.0.0.1", 12000))

# 접속이 되면 recv_message 함수를 쓰레드로 생성 합니다.
# recv_message 함수에 인자값을 넘기려면 args=튜플 형태로 인자값을 넘길 수 있습니다.
# 여기서는 접속된 socket 을 넘기기 위해 sock을 주었습니다.
th = Thread(target=recv_message, args=(sock, ))

# 프로그램이 종료하면 쓰레드도 종료되게 하기 위해 daemon=True 설정합니다.
th.daemon = True

# 쓰레드 시작
th.start()

# 입력 루프
while True:
    msg = input("입력: ")
    # 입력된 내용을 접속된 소켓을 통해 전송합니다.
    sock.send(msg.encode())
    
    # 입력된 내용이 /bye 면 루프를 탈출합니다.
    if msg == "/bye":
        break

# 접속된 소켓을 닫습니다.
sock.close()